#### 거리기반 K-Means의 문제점

* K-Means는 특정 중심점을 기반으로 거리적으로 퍼져있는 데이터 세트에 군집화를 적용하면 효율적 하지만 K-Means는 이러한 데이터 분포를 가지지 않는 데이터 세트에 대헤서는 효율적인 군집화가 어려움

## GMM(Gaussian Mixture Model) 개요

* GMM 군집화는 군집화를 적용하고자 하는 데이터가 여러 개의 다른 가우시안 분포(Gaussian Distribution)를 가지는 모델로 가정하고 군집화를 수행합니다.
* 가령 1000개의 데이터 세트가 있다면, 이를 구성하는 여러 개의 정규 분포 곡선을 추출하고, 개별 데이터가 이 중 어떤 정규 분포에 속하는지 결정하는 방식입니다.

### GMM 모수(Parameter) 추정

* GMM 모수 추정은 개별 정규 분포들의 평균과 분산, 그리고 데이터가 특정 정규 분포에 해당될 확률을 추정하는 것입니다.

### GMM 모수 추정을 위한 EM(Expectation and Maximization)

* Expectation : 개별 데이터 각각에 대해서 특정 정규 분포에 소속될 확률을 구하고, 가장 높은 확률을 가진 정규 분포에 소속(최초시에는 데이터들을 임의로 특정 정규 분포로 소속)

* Maximization : 데이터들이 특정 정규분포로 소속되면 다시 해당 정규분포의 평균과 분산을 구함. 해당 데이터가 발견될 수 있는 가능도를 최대화(Maximum likelihood) 할 수 있도록 평균과 분산(모수)를 구함


* 개별 정규분포의 모수인 평균과 분산이 더 이상 변경되지 않고, 각 개별 데이터들이 이전 정규 분포 소속이 더 이상 변경되지 않으면 그것으로 최종 군집화를 결정하고 그렇지 않으면 계속 EM 반복을 수행
    * EM : Expectation, Maximization

### 사이킷런 GaussianMixture

* 사이킷런은 GMM군집화를 위해 GaussianMixture 클래스를 제공
* GaussianMixture 클래스의 주요 생성자 파라미터는 n_components이며 이는 Mixture Model의 개수, 즉 군집화 개수를 의미